In [1]:
import pandas as pd
import numpy as np
import re 
import nltk 

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer
from clustering.clustering import ClusteringMethod
from sklearn.feature_extraction.text import CountVectorizer
from keybert import KeyBERT

import torch
torch.cuda.is_available()

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/

True

In [10]:
##### PREPROCESS ##########

df = pd.read_excel("/media/cattiaux/DATA/Wassati/team_data/lp3/Umfrage ZSP 2022 - Coop Gruppe(1-3749).xlsx", engine='openpyxl')
# Rename columns for readability
df = df.rename(columns={
    'Was zeichnet nach Ihren Erfahrungen einen guten Vorgesetzten / eine gute Vorgesetzte aus und was erwarten Sie von ihm respektive ihr? (Stichworte oder Fliesstext)': 'Comments'})
# df['year'] = pd.DatetimeIndex(df["Startzeit"]).year
# Replace 'Deutsch' with 'de'
df['Language'] = df['Language'].replace('Deutsch', 'de')
# Replace 'Français (France)' with 'fr'
df['Language'] = df['Language'].replace('Français (France)', 'fr')
# Replace 'English (United Kingdom)' with 'en'
df['Language'] = df['Language'].replace('English (United Kingdom)‎', 'en')
# Replace 'Italiano' with 'it'
df['Language'] = df['Language'].replace('Italiano', 'it')
# df.to_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments.csv", header=True, index=False)

df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/hlt_open_answer_with_language.csv")
df2 = df2.rename(columns={'languages':'Language', 'answer':"Comments"})

df_merge = df.append(df2, ignore_index=True)

# df_merge.to_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments_merge.csv", header=True, index=False)
# df_merge = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments_merge.csv")

# Filter rows where the 'Language' column is in the specified values
desired_languages = ['en', 'it', 'fr', 'de']
df_merge = df_merge[df_merge['Language'].isin(desired_languages)]

# Replace newline characters with a space
df_merge['Comments'] = df_merge['Comments'].str.replace('\n', '. ')

df_merge = df_merge.dropna(subset=['Comments'])

In [2]:
# # Mapping of language codes to full language names
# language_mapping = {
#     'fr': 'French',
#     'de': 'German',
#     'es': 'Spanish',
#     'it': 'Italian'
# }

# # Add the prefix based on the language value
# df_merge['comment_to_translate'] = df_merge.apply(lambda row: f"Translate {language_mapping.get(row['Language'], row['Language'])} to English: {row['Comments']}" if row['Language'] != 'en' else row['Comments'], axis=1)

In [11]:
###################### MAUVAISE TRADUCTION ###

# traduire iloc 4113 : A GOOD CHIEF IS A PERSON WHO HAS A GREAT SENSE OF LISTENING AND WHO KNOWS HOW TO APPRECIATE THE NEEDS OF HIS COLLABORATORS AND RESPOND TO THEM WHILE TAKING INTO ACCOUNT THE REQUIREMENTS OF COMPANY POLICY. IT'S ALSO SOMEONE WHO KNOWS MY JOB AND NOT MY TASKS, AND WHO CAN UNDERSTAND MY EXPECTATIONS AND NEEDS FOR DEVELOPMENT. IT'S ALSO SOMEONE WHO KNOWS HOW TO CONGRATULATE AND REPRIMAND, TO SET THE RIGHT PACE AND GIVE RESPONSIBILITY TO EMPLOYEES. 
# traduire iloc 1505 : Give trust (if it is not negatively exploited); trust high performance and hand over responsibility; support the employee (stand by him/her, also towards superiors); stand by his/her word; offer support in case of problems; give feedback (especially positive but also potential for improvement) and, where appropriate, tips for improvement; be a role model with regard to work commitment, work ethic, behavior and dealing with others; show interest in the employees (e.g. ask in the EC how things are going); support further development if possible.
# traduire iloc 2742 : Coaching, listening well, supporting and encouraging employees, networked thinking
# traduire iloc 444 : Role model, help in Rayon, to all employees, loyal, ensure information
# traduire iloc 4703 : A good boss is: Resilient Close to the employees Has time for his team Addresses good and bad issues directly Objective or treats all team members equally
# traduire iloc 634 : Freedom for self-development, but also certain guard rails. Objective and critical exchange on both sides.  Clear communication, not one way today, one way tomorrow.  Just leave it simple.
# traduire iloc 3030 : Expect him to be there when I need him.  A good boss must be correct, experienced and a leader.
# traduire iloc 2306 : Openness and honesty in communication, support with problems before and/or with customers, support with issues related to the Head of Customer, and not always having to hear ‘yes, that’s just how it is,’ but also expressing what is expected from the front to the Head of Customer
# traduire iloc 280 : Overall situation in view, fairness, respect, clear line, human, direct and timely conversation, if something does not go as it should, where feasible give freedom/responsibility to employees, coach in certain situations
# traduire iloc 682 : Role model, loyal, human
# traduire iloc 5470 : Good bosses take employees seriously and show them how. Bosses have foresight and strategic thinking and don't just dwell on the small details, but also need humility towards detail. Bosses need distance to subordinate employees without being arrogant, aloof and distant from them. Good bosses should be able to accept criticism and recognize narcissistic slights in themselves.
# traduire iloc 2177 : A good supervisor should lead with intelligence, not with empty talk. And why give an explanation ?
# traduire iloc 5847 : -Steers the employee in the desired direction in a way that he/she can understand -Can express him/herself clearly and comprehensibly -Does not play games with the employee -Is willing to listen and engage with the employee. -A feeling for the employees and the ability to manage them correctly and fairly is more important than a great deal of specialist knowledge. -Delivers the tasks to be performed, clearly defined and with the necessary responsibilities and competencies. Allows new or different solutions to be proposed
# traduire iloc 5305 (typo gleih -> gleich) : that we are all treated EQUALLY by him/her ... human, fair, correct that i can talk to him/her in case of problems (no matter work or private) that he/she can listen and give information that can laugh and support who is needed that lie the whole team and not only some (good friend) is doing well at work ( we are a team) 
# traduire iloc 3002 : Recognition of performance, fair, positive
# traduire iloc 3904 : Humanity, clear orders, helps to solve problems, good communication
# traduire iloc 4401 : Human and professionally competent
# traduire iloc 6280 : Balance between AUTHORITY AND COLLABORATION, PROFESSIONAL, COMMUNICATIVE, RESPECTFUL, CLEAR AND ACCURATE, CAN GIVE A GOOD SPEECH, DOWN TO EARTH, CAREFUL APPEARANCE, RELIABLE, HUMAN, RISK-READY, CONVINCING, SELF-AWARE 
# traduire iloc 1829 : A good manager is someone who is transparent, correct, exemplary, committed and a leader. (ménager au lieu de manager, de au lieu de fr)
# traduire iloc 3306 : A good manager sets a clear course for success by adapting to the profiles of his subordinates. I expect him to give me his trust and candor when necessary. 


# interest in MAB show -> show interest in employees
# Fach- & Sozialkompetenz -> Professional and social competence
# Thus the leader gets the "Sörgeli u Ängstli" with. -> This way, the leader becomes aware of the team’s concerns and anxieties.
# the 5s stand. -> things slide.
# even a 5 straight -> even let things slide

########################## SIGLES ###

# MAs -> employees (Mitarbeiter)
# MA -> employee (Mitarbeiter)
# MT -> management team
# vg -> supervisor / manager (Vorgesetzter)
# VST -> managers /supervisors (Vorgesetzter)
# SBB -> State Library Berlin (Staatsbibliothek zu Berlin)
# ggü -> towards (gegenüber)
# AKV -> Tasks, Competencies, Responsability (Aufgaben, Kompetenzen, Verantwortung)
# Stv -> representative (stellvertretend)
# GFs -> managers (Geschäftsführer)
# GF -> managers (Geschäftsführer)
# abbot -> department (Abteilung)
# Abt -> department (Abteilung)
# PK Kostenrechnung -> process cost calculation (Prozesskostenrechnung)
# SK -> expertise (Sachkompetenz)
# AL -> head of department (Abteilungleiter) (AL/RL dans le texte)
# RL -> Regional manager (Regionalleiter)
# PEP -> Energy
# AG -> employer (Arbeitgeber)
# AN -> employee (Arbeitnehmer)
# A + O -> Alpha and Omega
# KVP -> Continual Improvement Process (CIP) (Kontinuierlicher Verbesserungsprozess)
# Bzw. -> repectively / or (beziehungsweise)
# GL/GV -> gender relations (Geschlechtverkehr)
# FL -> branch manager (Filialleiter)
# RVL -> sales manager (Verkaufsleiter)
# SZB -> Biel Hospital Center (Spitalzentrum Biel)

# drh -> human resources manager
# rh -> human resources
# HR -> human resources
# dg -> general manager

# cla -> Contributor License Agreement
# HC -> Head Coach
# MBO -> Management By Objectives
# VC -> Venture Capitalist

# pdv -> point of sale (point de vente / punto di vendita)

####################### ERREURS DE TYPO ###

# mangment -> management
# methrises -> masters (métrise au lieu de maîtrise et donc traduit methrises)
# mangment of equpy -> team management (managment d'équpi)
# Vobild -> Role model / leading (Vorbild)
# Vorbil, -> Role model, (Vorbild)
# vorblidfunktion -> role model function
# Nn -> doesn't (non)
# suvisory -> sufficient
# saoirs -> knowledge (savoirs)
# Preceive with -> Leading by 
# iformations -> information
# SMARTE -> SMART
# challengen -> challenge
# Mutivation -> motivation
# Autenthcity -> Authenticity
# KLARE KOMMUNIKATIN -> Clear communicator
# mitarbiters -> employees
# sitautions -> situations
# esprit -> spirit (ésprit au lieu de esprit)
# Erliche -> honest (erliche au lieu de ehrliche)
# Ehrichness -> honesty
# ehrich -> honest
# bos -> boss
# a aller -> an ally (typo allier au lieu de allié)
# Erlichkeit -> honesty (typo ehrlichkeit)
# Iel -> He (typo il)
# pk -> why
# makromanagemen -> macromanagement (typo)
# Teli -> part (typo for Teil)
# Fuehrundsfaehigket -> Leadership skills (Führungsfähigkeit)
# fuehrungsbereitschaft -> willingness to lead (führungsbereitschaft)
# delegateing -> delegating
# metered -> measured
# orchestieren -> orchestrating
# yaka -> all you have to do is / it's just a matter of (il n'y a qu'à)
# aurat -> accurate (akurat au lieu de akkurat)
# examplearity -> examplarity
# dialigo -> dialogue (dialogo)
# besoi,n -> need
# fairnaiss -> fairness
# qqn -> someone,
# accessibilita -> accessibility
# deligate -> delegate

#################### OTHER REPLACMENT ###

# laisse faire -> laisser faire
# laisser-faire -> laisser faire
# laissez-faire -> laisser faire
# selfmanagement -> self management
# self-management -> self management
# Mut -> Courage
# Menschenfreund -> philanthropist / humanitarian
# ehrlich -> honest
# fehl -> wrong
# CCC: Comandare, Controllare, Correggere -> CCC: Command, Control, Correct.
# kritik -> Specialist knowledge
# chefin -> boss
# "LEITET" -> LEADS
# "LEITEN" -> suffer
# ziel -> target
# prudentia -> prudence
# 1x1 -> one on one
# clairvoyance -> clear-sightedness
# Vorbildsfunktion -> Role model function
# merci -> thank you
# troupes -> troops
# leist -> performance
# rigueur -> rigor
# lob -> praise
# clientele -> customers
# Gelichbehandlungen -> treat the same
# "wischi" waschi -> wishy-washy 
# agreementrity -> agreement
# patron -> boss
# Führungsbereitung -> leadership preparation

In [32]:
# Find rows that contain specific word
pd.set_option('display.max_colwidth', None)
df_translated[df_translated['translated_comments'].str.contains("fhsfg", case=False)] 

# df_translated.iloc[5513:5515]


,ID,Startzeit,Fertigstellungszeit,E-Mail,Name,Language,Comments,is_boss,coach,comment_to_translate,translated_comments
5518,NaN,NaN,NaN,NaN,NaN,de,fhsfg,0.0,KV Business School Zürich AG,Translate German to English: fhsfg,fhsfg


In [16]:
update_rows = {
    4113:"""A GOOD CHIEF IS A PERSON WHO HAS A GREAT SENSE OF LISTENING AND WHO KNOWS HOW TO APPRECIATE THE NEEDS OF HIS COLLABORATORS AND RESPOND TO THEM WHILE TAKING INTO ACCOUNT THE REQUIREMENTS OF COMPANY POLICY. IT'S ALSO SOMEONE WHO KNOWS MY JOB AND NOT MY TASKS, AND WHO CAN UNDERSTAND MY EXPECTATIONS AND NEEDS FOR DEVELOPMENT. IT'S ALSO SOMEONE WHO KNOWS HOW TO CONGRATULATE AND REPRIMAND, TO SET THE RIGHT PACE AND GIVE RESPONSIBILITY TO EMPLOYEES.""",
    1505:"""Give trust (if it is not negatively exploited); trust high performance and hand over responsibility; support the employee (stand by him/her, also towards superiors); stand by his/her word; offer support in case of problems; give feedback (especially positive but also potential for improvement) and, where appropriate, tips for improvement; be a role model with regard to work commitment, work ethic, behavior and dealing with others; show interest in the employees (e.g. ask in the EC how things are going); support further development if possible.""",
    2742:"""Coaching, listening well, supporting and encouraging employees, networked thinking""",
    444:"""Role model, help in Rayon, to all employees, loyal, ensure information""",
    4703:"""A good boss is: Resilient Close to the employees Has time for his team Addresses good and bad issues directly Objective or treats all team members equally""",
    634:"""Freedom for self-development, but also certain guard rails. Objective and critical exchange on both sides.  Clear communication, not one way today, one way tomorrow. Just leave it simple.""",
    3030:"""Expect him to be there when I need him.  A good boss must be correct, experienced and a leader.""",
    2306:"""Openness and honesty in communication, support with problems before and/or with customers, support with issues related to the Head of Customer, and not always having to hear "yes, that’s just how it is" but also expressing what is expected from the front to the Head of Customer""",
    280:"""Overall situation in view, fairness, respect, clear line, human, direct and timely conversation, if something does not go as it should, where feasible give freedom/responsibility to employees, coach in certain situations""",
    682:"""Role model, loyal, human""",
    5470:"""Good bosses take employees seriously and show them how. Bosses have foresight and strategic thinking and don't just dwell on the small details, but also need humility towards detail. Bosses need distance to subordinate employees without being arrogant, aloof and distant from them. Good bosses should be able to accept criticism and recognize narcissistic slights in themselves.""",
    2177:"""A good supervisor should lead with intelligence, not with empty talk. And why give an explanation ?""",
    5847:"""-Steers the employee in the desired direction in a way that he/she can understand -Can express him/herself clearly and comprehensibly -Does not play games with the employee -Is willing to listen and engage with the employee. -A feeling for the employees and the ability to manage them correctly and fairly is more important than a great deal of specialist knowledge. -Delivers the tasks to be performed, clearly defined and with the necessary responsibilities and competencies. Allows new or different solutions to be proposed""",
    5305:"""that we are all treated EQUALLY by him/her ... human, fair, correct that i can talk to him/her in case of problems (no matter work or private) that he/she can listen and give information that can laugh and support who is needed that lie the whole team and not only some (good friend) is doing well at work ( we are a team) """,
    3002:"""Recognition of performance, fair, positive""",
    3904:"""Humanity, clear orders, helps to solve problems, good communication""",
    4401:"""Human and professionally competent""",
    6280:"""Balance between AUTHORITY AND COLLABORATION, PROFESSIONAL, COMMUNICATIVE, RESPECTFUL, CLEAR AND ACCURATE, CAN GIVE A GOOD SPEECH, DOWN TO EARTH, CAREFUL APPEARANCE, RELIABLE, HUMAN, RISK-READY, CONVINCING, SELF-AWARE""",
    1829:"""A good manager is someone who is transparent, correct, exemplary, committed and a leader.""",
    3306:"""A good manager sets a clear course for success by adapting to the profiles of his subordinates. I expect him to give me his trust and candor when necessary.""",
    5945:"""role model function in terms of professional competence and social skills. A boss should encourage employees""",
    3706:"A clear vision and exemplary behavior",
    5695:"""-Good supervisors fulfill their management tasks (in particular: planning, ensuring goals are achieved, having processes under "control", making decisions, communicating). -Their professional competence enables them to make appropriate decisions and communicate. -Their social competence enables them to understand employees as people, to respond to them and to inspire/motivate them for their tasks."""
    }

replacements_sentence = {
    "interest in MAB show" : "show interest in employees",
    "Fach- & Sozialkompetenz" : "Professional and social competence",
    """Thus the leader gets the "Sörgeli u Ängstli" with.""" : """This way, the leader becomes aware of the team’s concerns and anxieties.""",
    "the 5s stand." : "things slide.",
    "even a 5 straight" : "even let things slide",
    "Unterstüzung anbieten":"offer support"
}

replacements_sigle = {
    " MAs ":" employee ",
    " MA ":" employee ",
    " MA,":" employee,",
    " MT ":" management team ",
    " vgs ":"manager",
    " vg ":" manager ",
    " vsts":" manager",
    " VST":" manager",
    " SBB":" State Library Berlin",
    " ggü":" towards",
    "AKV":"Tasks, Competencies, Responsability",
    "Stv":"representative",
    " GFs":" manager",
    " GF":" manager",
    "abbot":"department",
    " Abt":" department",
    "PK Kostenrechnung":"process cost calculation",
    " SK ":" expertise ",
    " AL ":" head of department ",
    " AL/":" head of department/",
    "RL ":"regional manager ",
    " PEP":" energy",
    " AG ":" employer ",
    " AN ":" employee ",
    "A + O":"alpha and omega",
    "KVP":"Continual Improvement Process (CIP)",
    "Bzw.":"or",
    "GL/GV":"gender relations",
    " FL ":" branch manager ",
    " RVL":" sales manager",
    "SZB":"Biel Hospital Center",
    " drh":" human resoures manager",
    " rh":" human resources",
    " HR":" human resources",
    " dg":" general manager",
    " cla ":" Contributor License Agreement ",
    " HC ":" head coach ",
    " pdv":" point of sale",
    " mbo":" Management By Objectives",
    " vc":" venture capitalist"
}

replacements_typo = {
    "methrises":"masters",
    "mangment of equpy":"team management",
    "mangment":"management",
    "Vobild":"role model",
    "Vorbil,":"role model",
    "vorbldfunction":"role model function",
    "vorblidfunktion":"rol model function",
    " Nn ":" does not ",
    "suvisory":"sufficient",
    "saoirs":"knowledge",
    "Preceive with":"leading by",
    "iformations":"information",
    "SMARTE ":"SMART ",
    "challengen":"challenge",
    "Mutivation":"motivation",
    "Autenthcity":"authenticity",
    "KLARE KOMMUNIKATIN":"clear communicator",
    "mitarbiters":"employees",
    "sitautions":"situations",
    "esprit ":"spirit ",
    "Erliche ":"honest ",
    "Ehrichness":"honesty",
    "ehrich":"honest",
    " bos ":" boss ",
    "a aller":"an ally",
    "Erlichkeit":"honesty",
    " Iel ":" He ",
    " pk ":" why ",
    "makromanagemen":"macromanagement",
    "Teli ":"part ",
    "Fuehrundsfaehigket":"Leadership skills",
    "fuehrungsbereitschaft":"willingness to lead",
    "delegateing":"delegating",
    "metered":"measured",
    "orchestieren":"orchestrating",
    "yaka":"it's just a matter of",
    "aurat ":"accurate ",
    "examplearity":"examplarity",
    "dialigo":"dialogue",
    "besoi,n":"need",
    "fairnaiss":"fairness",
    "qqn":"someone",
    "accessibilita":"accessibility",
    "deligate":"delegate",
    "cheffs":"bosses"
}

replacements_others = {
    "Mut ":"courage ",
    "Menschenfreund":"philanthropist",
    "ehrlich":"honest",
    "fehl":"wrong",
    "CCC: Comandare, Controllare, Correggere":"CCC: Command, Control, Correct",
    "kritik":"specialist knowledge",
    "chefin":"boss",
    "LEITET":"leads",
    "leiten":"suffer",
    "ziel":"target",
    "prudentia":"prudence",
    "1x1":"one on one",
    "clairvoyance":"clear-sightedness",
    "Vorbildsfunktion":"role model function",
    "merci":"thank you",
    "leist":"performance",
    "rigueur":"rigor",
    "Visionär":"visionary",
    "Zielstrebig":"determined",
    "Offen ":"open ",
    "laisser-faire":"laisser faire",
    "laissez-faire":"laisser faire",
    "laisse faire":"laisser faire",
    "lob ":"praise ",
    "clientele":"customer",
    "Gelichbehandlungen":"treat the same",
    "'wischi' waschi":"wishy washy",
    "wishy-washy":"wishy washy",
    "agreementrity":"agreement",
    "patron":"boss",
    "Führungsbereitung":"leadership preparation",
    "orientedness":"oriented",
    "openness":"open",
    "fairness":"fair",
    "helpiveness":"helpfulness",
    "exemplaryness":"exemplary",
    "exemplaryity":"exemplarity",
    "exemplararity":"exemplarity",
    "exemplarité":"exemplarity",
    "value-oriented":"value oriented",
    "Teamable":"team player",
    "teamplayer":"team player",
    "long-term":"long term",
    "superiors":"superior",
    "chefs":"chef",
    "collaborators":"collaborator",
    "ears":"ear",
    "criticisms":"criticism",
    "freedoms":"freedom",
    "competences":"competence",
    "teamness":"team spirit",
    "loyality":"loyalty",
    "dealings":"treatment",
    "motivative":"motivate",
    "leaven":"calm",
    "sparing":"sparring",
    "manages":"manage",
    "bosses":"boss",
    "supervisors":"supervisor",
    "coaches":"coach",
    "employees":"employee",
    "employing":"employs",
    "motivates":"motivate",
    "communicates":"communicate",
    "trusting":"trust",
    "managers":"manager",
    "communicators":"communicator",
    "leads":"lead",
    "commissions":"commission",
    "leaders":"leader",
    "questions":"question",
    "colleagues":"colleague",
    "motivating":"motivate",
    "communications":"communication",
    "listening":"listen",
    "listened":"listen",
    "partners":"partner",
    "communicating":"communicate",
    "meetings":"meeting",
    "communicated":"communicate",
    "customers":"customer",
    "keeping":"keep",
    "waiting":"wait",
    "understands":"understand",
    "answered":"answer",
    "participates":"participate",
    "trusts":"trust"
}

ngrams_list = ["laisser faire", "Anselm Grün", "specialist knowledge", "dossier knowledge", "force majeure", "thank you", "professional competence", "human resources", "branch manager", "sales manager", "lao tzeu", "management team", "wishy washy", "venture capitalist", "self management", "ear criticism", "supports problems", "decisions order", "lead person","value oriented", "distributed tasks", "analytical mind", "labour market", "confidentiality information", "resetting ego", "team player", "team organization", "cooperative spirit", "team spirit", "trust team", "open ear","role model", "make decisions", "high level", "clear rules", "clear vision", "clear line", "clear communication", "honest communication", "transparent communication", "clear goals", "clear orders", "clear guidelines", "clear objectives", "clear decisions", "clear instructions", "clear tasks", "clear targets", "social competence", "team collaboration", "team members", "value oriented", "constructive feedback", "open communication", "goes ahead", "stands up", "equal treatment", "long leash", "short leash", "eye level", "professionally competent", "socially competent", "work organization", "target agreement","soial skills", "communication skills", "professional skills", "leadership skills", "soft skills", "observation skills", "team skills", "difficult situations", "personal development", "professional development", "career development", "further development", "keep promises", "respectful treatment", "remain calm", "stay calm", "back cover", "blind spot", "front line", "good example", "good leader","good supervisor","good manager","good boss", "good superior", "family member", "sparring partner", "exemplary behavior", "Continual Improvement Process", "one on one", "process cost calculation", "Biel Hospital Center", "Management By Objectives", "point of sale", "strengths and weaknesses", "climate of trust", "people with disabilities", "long term vision"]

more_stopwords = ["MMMM","style","good boss","good leader","good supervisor","good superior","good manager","good coach","good","boss","leader","manager","superior", "supervisor","person"]
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(more_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cattiaux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
df_translated = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments_merge_translated.csv")

# List of rows to be excluded
rows_to_exclude = [2466,3191,3366,3976,4015,6562,6614]
rows_to_exclude2 = [2116,5351,7055,5334,4566,4232,6103,6104,6105,6109,6110,3776,4397,5513]

# Selecting rows that are not in the list
df_translated = df_translated[~df_translated.index.isin(rows_to_exclude)]

# Update the "translated_comments" column using the dictionary
for iloc, new_comment in update_rows.items():
    df_translated.at[df_translated.index[iloc], 'translated_comments'] = new_comment

# Replace the words in the 'translated_comments' column
# Create a combined regex pattern from the dictionary keys
df_translated['translated_comments'] = df_translated['translated_comments'].replace(replacements_sentence, regex=True)
# regex_pattern = '(' + '|'.join(re.escape(key) for key in replacements_sigle.keys()) + ')'
# df_translated['translated_comments'] = df_translated['translated_comments'].str.replace(regex_pattern, lambda m: replacements_sigle.get(m.group(0).lower(), m.group(0)), flags=re.IGNORECASE, regex=True)
# # df_translated['translated_comments'] = df_translated['translated_comments'].str.replace(regex_pattern, replacement_function, flags=re.IGNORECASE, regex=True)
# regex_pattern = '(' + '|'.join(re.escape(key) for key in replacements_typo.keys()) + ')'
# df_translated['translated_comments'] = df_translated['translated_comments'].str.replace(regex_pattern, lambda m: replacements_typo.get(m.group(0).lower(), m.group(0)), flags=re.IGNORECASE, regex=True)
# regex_pattern = '(' + '|'.join(re.escape(key) for key in replacements_others.keys()) + ')'
# df_translated['translated_comments'] = df_translated['translated_comments'].str.replace(regex_pattern, lambda m: replacements_others.get(m.group(0).lower(), m.group(0)), flags=re.IGNORECASE, regex=True)


# Function to replace keys in the text with their corresponding values
def replace_keys(text, dico):
    pattern = '(' + '|'.join(re.escape(key) for key in dico.keys()) + ')'
    def match_case(match):
        key = match.group(0).lower()
        return dico.get(key, key)
    return re.sub(pattern, match_case, text, flags=re.IGNORECASE)

# Apply the function to the 'translated_comments' column
df_translated['translated_comments'] = df_translated['translated_comments'].apply(lambda text: replace_keys(text, replacements_sigle))
df_translated['translated_comments'] = df_translated['translated_comments'].apply(lambda text: replace_keys(text, replacements_typo))
df_translated['translated_comments'] = df_translated['translated_comments'].apply(lambda text: replace_keys(text, replacements_others))

df_translated = df_translated[~df_translated.index.isin(rows_to_exclude2)]

# Filter out the rows that contain any of the specified tests
tests_to_exclude = '|'.join(['Test 1', 'Test 2', 'Test 3', 'Test 4'])
df_translated = df_translated[~df_translated['translated_comments'].str.contains(tests_to_exclude, case=False, na=False)]

# df_translated.iloc[3306:3307]

In [ ]:
########## CHECK KEYWORDS WITH KEYERT ################

# model_name = "all-MiniLM-L6-v2"

# # Initialize KeyBERT with the specified model
# kw_model = KeyBERT(model_name)

# # Extract the top n unigrams for each document
# df_translated['top_unigrams'] = df_translated['translated_comments'].apply(
#     lambda doc: kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words='english', 
#                                           use_mmr=True, top_n=4)
# )

# # To get a list of unique unigrams across all documents
# unique_unigrams = set()
# df_translated['top_unigrams'].apply(lambda keywords: unique_unigrams.update([word for word, _ in keywords]))

# # Convert the set to a list
# vocabulary_list = list(unique_unigrams)
# vocabulary_list

In [33]:
from collections import Counter
from nltk.util import ngrams

##### CHECK LIST BEST NGRAMS ################

# Function to generate n-grams from a list of tokens
def generate_ngrams(tokens, n):
    return list(ngrams(tokens, n))

# Choose the value of n for n-grams (e.g., 2 for bigrams, 3 for trigrams, etc.)
n = 2

# Generate n-grams for each row in the DataFrame
df_translated['ngrams'] = df_translated['translated_comments'].apply(lambda x: generate_ngrams(x.split(), n))

# Flatten the list of n-grams and count occurrences
ngram_counts = Counter([ngram for sublist in df_translated['ngrams'] for ngram in sublist])

# Create a DataFrame with n-grams and their frequencies
ngram_df = pd.DataFrame(ngram_counts.items(), columns=['Ngram', 'Frequency']).sort_values(by='Frequency', ascending=False)
# ngram_df.iloc[0:50]

# Function to check if the specified word is in the n-gram tuple
def contains_word(ngram_tuple, word, position=None):
    # Convert the tuple to lowercase to perform case-insensitive comparison
    ngram_tuple_lower = tuple(map(str.lower, ngram_tuple))
    word_lower = word.lower()
    # If position is specified, check the specific index
    if position is not None:
        return ngram_tuple_lower[position] == word_lower if position < len(ngram_tuple_lower) else False
    # If position is not specified, check if the word is anywhere in the tuple
    return word_lower in ngram_tuple_lower

# ngram_df[ngram_df['Ngram'].apply(lambda x: contains_word(x, "team"))][:50]

# back,


In [31]:
from vocabulary.vocabulary import VocabularyCreator

df_translated = df_translated.rename(columns={
    'translated_comments': 'processed_data'})

model_name = "all-MiniLM-L6-v2"

keybert_kwargs={
    "top_n": 5,
    "use_mmr": True,
    "stop_words": "english",
    "keyphrase_ngram_range": (1, 1),
    "nr_candidates": 12
}

vocabulary_creator = VocabularyCreator(model_name,ngrams_list,**keybert_kwargs)
vocabulary_list = vocabulary_creator.keybert_vocabulary(df_translated)
len(vocabulary_list)

3412

In [54]:
######### MODULE TRANSLATE ###########

# Use a pipeline as a high-level helper
from transformers import pipeline

# Set the device to GPU if available, otherwise use CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create translation pipelines for each language pair
german_to_english = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en", max_length=1350)#, device=device)
french_to_english = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", max_length=1350)#, device=device)
italian_to_english = pipeline("translation", model="Helsinki-NLP/opus-mt-it-en", max_length=1350)#, device=device)

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [131]:
######### MODULE TRANSLATE ###########

########## TEST traduction AVEC mbart MODEL ###########

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# # Define the manual mapping of language codes
# language_mapping = {
#     "fr": "fr_XX",  # French
#     "de": "de_DE",  # German
#     "it": "it_IT",  # Italian
#     # Add more mappings as needed
# }

# def translate_comment(comment, language_code):
#     if language_code == "en":
#         # No need to translate English comments
#         return comment
#     else:
#         src_lang = language_mapping.get(language_code, None)
#         if src_lang is None:
#             raise ValueError(f"Unsupported language code: {language_code}")
        
#         tgt_lang = "en_XX"
#         encoded_comment = tokenizer(comment, return_tensors="pt")
#         generated_tokens = model.generate(
#             **encoded_comment,
#             forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang]
#         )
#         translated_comment = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
#         return translated_comment
    
# # df_merge["Translated_Comments"] = df_merge.apply(lambda row: translate_comment(row["Comments"], row["Language"]), axis=1)
# df_merge.iloc[1100:1150].apply(lambda row: translate_comment(row["Comments"], row["Language"]), axis=1)


In [156]:
######## TRANSLATION MODELS TO ENGLISH ##########

from semantic_text_splitter import TextSplitter

def translate_comment(row):
    comment = row['Comments']
    lang = row['Language']
    
    if lang == 'fr':
        # Translate French to English
        return french_to_english(comment)[0]['translation_text']
    elif lang == 'de':
        # Translate German to English
        return german_to_english(comment)[0]['translation_text']
    elif lang == 'it':
        # Translate Italian to English
        return italian_to_english(comment)[0]['translation_text']
    else:
        # No translation needed for English comments
        return comment

def translate_comment_by_seq(row):
    comment = row['Comments']
    lang = row['Language']
    
    # Split the comment into sentences (adjust the splitting logic)
    def split_sentences(text):
        sentences = []
        current_sentence = ""
        for token in text.split(". "):
            if token[0].isdigit() and len(token) > 1 and token[1] == ".":
                # Append the current sentence and start a new one
                sentences.append(current_sentence.strip())
                current_sentence = token
            else:
                current_sentence += ". " + token
        sentences.append(current_sentence.strip())  # Append the last sentence
        return sentences
    
    sentences = split_sentences(comment)
    sentences = comment.split(". ")

    # Initialize an empty list to store translated sentences
    translated_sentences = []

    # Translate each sentence separately
    for sentence in sentences:
        if lang == 'fr':
            # Translate French to English
            try:
                translated_sentence = french_to_english(sentence)[0]['translation_text']
            except IndexError:
                translated_sentence = sentence
        elif lang == 'de':
            # Translate German to English
            try:
                translated_sentence = german_to_english(sentence)[0]['translation_text']
            except IndexError:
                translated_sentence = sentence
        elif lang == 'it':
            # Translate Italian to English
            try :
                translated_sentence = italian_to_english(sentence)[0]['translation_text']
            except IndexError:
                translated_sentence = sentence
        else:
            # No translation needed for English comments
            translated_sentence = sentence

        # Append the translated sentence to the list
        translated_sentences.append(translated_sentence)

    # Combine the translated sentences
    full_translation = ". ".join(translated_sentences)
    return full_translation

def translate_comment_by_chunks(row, chunk_size=400):
    comment = row['Comments']
    lang = row['Language']

    # Initialize the TextSplitter with the desired chunk size
    splitter = TextSplitter(chunk_size)

    # Split the comment into chunks
    chunks = splitter.chunks(comment)

    # Initialize an empty list to store translated chunks
    translated_chunks = []

    # Translate each chunk separately
    for chunk in chunks:
        if lang == 'fr':
            # Translate French to English
            try:
                translated_chunk = french_to_english(chunk)[0]['translation_text']
            except IndexError:
                translated_chunk = chunk
        elif lang == 'de':
            # Translate German to English
            try:
                translated_chunk = german_to_english(chunk)[0]['translation_text']
            except IndexError:
                translated_chunk = chunk
        elif lang == 'it':
            # Translate Italian to English
            try:
                translated_chunk = italian_to_english(chunk)[0]['translation_text']
            except IndexError:
                translated_chunk = chunk
        else:
            # No translation needed for English comments
            translated_chunk = chunk

        # Append the translated chunk to the list
        translated_chunks.append(translated_chunk)

    # Combine the translated chunks
    full_translation = " ".join(translated_chunks)
    return full_translation

# Apply translation to each row
df_merge['translated_comments'] = df_merge.apply(translate_comment_by_chunks, axis=1)
# df_merge[2000:3000].apply(translate_comment_by_chunks, axis=1)

# df_merge.to_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments_merge_translated.csv", header=True, index=False)

In [36]:
# Run BERTopic

clustering = ClusteringMethod(model_name)

bertopic_kwargs={
    "nr_topics":"auto",
    "ctfidf_model": ClassTfidfTransformer(reduce_frequent_words=True),
    "representation_model": [KeyBERTInspired(), MaximalMarginalRelevance(diversity=0.4)], # possible values : KeyBERTInspired() or MaximalMarginalRelevance(diversity=0.5)
    "top_n_words":10,
    # "min_topic_size":50,
    "calculate_probabilities": True
}

bertopic_kwargs['vectorizer_model'] = CountVectorizer(
                    vocabulary=vocabulary_list, 
                    stop_words=stopwords, 
                    lowercase=True, 
                    ngram_range=(1, 3)
                )

docs = df_translated["processed_data"].astype(str).tolist()

topics, probs, topic_model, embeddings = clustering.run_bertopic(
    df= df_translated,
    **bertopic_kwargs
    )

Batches:   0%|          | 0/228 [00:00<?, ?it/s]

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning:

divide by zero encountered in divide



In [88]:
# ADD NGRAM AFTER CREATION OF THE BERTOPIC MODEL
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
# topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

In [37]:
topic_model.visualize_topics()

In [191]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 17/17 [04:18<00:00, 15.18s/it]


In [38]:
topic_model.visualize_barchart(top_n_topics=50)
# 

In [42]:
def isValid_cpf(cpf):
    # Remove any formatting
    cpf = ''.join(filter(str.isdigit, cpf))
    
    # Check if CPF has 11 digits
    if len(cpf) != 11:
        return False
    
    # Check if CPF has all digits equal
    if cpf == cpf[0] * len(cpf):
        return False
    
    # Calculate first check digit
    first_check_sum = sum(int(digit) * weight for digit, weight in zip(cpf[:9], range(10, 1, -1)))
    first_check_digit = (first_check_sum * 10 % 11) % 10
    if int(cpf[9]) != first_check_digit:
        return False
    
    # Calculate second check digit
    second_check_sum = sum(int(digit) * weight for digit, weight in zip(cpf[:10], range(11, 1, -1)))
    second_check_digit = (second_check_sum * 10 % 11) % 10
    if int(cpf[10]) != second_check_digit:
        return False
    
    return True

isValid_cpf("11418074403")

def complete_cpf(incomplete_cpf, nb_possibilities=None):
    def calculate_check_digit(digits, position):
        sum_of_products = sum(int(digit) * weight for digit, weight in zip(digits, range(position, 1, -1)))
        check_digit = (sum_of_products * 10 % 11) % 10
        return str(check_digit)

    def generate_possibilities(cpf, missing_indices, current_index, possibilities):
        if current_index == len(missing_indices):
            # Only calculate check digits if they are missing (marked by asterisks)
            if cpf[9] == '*':
                cpf[9] = calculate_check_digit(cpf[:9], 10)
            if cpf[10] == '*':
                cpf[10] = calculate_check_digit(cpf[:10], 11)
            completed_cpf_str = ''.join(cpf)
            if isValid_cpf(completed_cpf_str) and completed_cpf_str not in possibilities:
                possibilities.append(completed_cpf_str)
            return

        for digit in '0123456789':
            cpf[missing_indices[current_index]] = digit
            generate_possibilities(cpf, missing_indices, current_index + 1, possibilities)

    # Remove formatting and find missing indices
    cpf = list(incomplete_cpf.replace('.', '').replace('-', ''))

    missing_indices = [i for i, digit in enumerate(cpf) if digit == '*']

    if len(cpf) != 11 or len(missing_indices) == 0:
        return []

    possibilities = []
    generate_possibilities(cpf, missing_indices, 0, possibilities)

    if nb_possibilities is not None:
        return possibilities[:nb_possibilities]
    return possibilities

complete_cpf('063.387.*47-0*')  # Returns all valid CPF completions

['06338734703', '06338764700']